In [91]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 打开浏览器时的相关配置，可以根据需求进行打开和关闭
options = Options()
# options.add_argument("--headless")       # 后台操作
# options.add_argument("--start-maximized")  # 启动时最大化窗口
# options.add_argument("--disable-blink-features=AutomationControlled")  # 使浏览器不显示自动化控制的信息
# options.add_argument("--disable-gpu")  # 禁用GPU硬件加速
# options.add_argument("--disable-infobars")  # 隐藏信息栏
# options.add_argument("--disable-extensions")  # 禁用所有扩展程序
# options.add_argument("--disable-popup-blocking")  # 禁用弹出窗口拦截
# options.add_argument("--incognito")  # 启动无痕模式
# options.add_argument("--no-sandbox")  # 关闭沙盒模式（提高性能）
# options.add_argument("--disable-dev-shm-usage")  # 使用/dev/shm分区以避免共享内存问题
# options.add_argument("--remote-debugging-port=9222")  # 启用远程调试端口
# 初始化 WebDriver，并传入 ChromeDriver Service
driver = webdriver.Chrome(options=options)

In [92]:
import pandas as pd

titles_df = pd.read_csv('./Data_TitleInfo.csv')
titles_ids = titles_df['title_ID'].map(lambda x: str(x).split('_')[1]).to_list()

In [93]:
titles_df.head()

,index,title_ID,score,knowledge,sub_knowledge
0,1,Question_2119,1,Linear Lists,Linked Lists
1,2,Question_2126,1,Linear Lists,Arrays
2,3,Question_1325,1,Linear Lists,Arrays
3,4,Question_2143,1,Stacks & Queues,Queues
4,5,Question_2142,1,Stacks & Queues,Stacks


In [94]:
# 登陆
driver.get("https://acm.sjtu.edu.cn/OnlineJudge/login?next=%2FOnlineJudge%2F")
username_input = driver.find_element(By.XPATH, "//input[@id='username']")
password_input = driver.find_element(By.XPATH, "//input[@id='password']")
username_input.send_keys("qishuobai")
password_input.send_keys("Bqs0208.")
password_input = driver.find_element(By.XPATH, "//input[@type='submit']").click()

In [95]:
import re
def validate_studentID(s):
    pattern = r'^\d{12}_.+$'
    return bool(re.match(pattern, s))

In [96]:
submit_record_df = pd.DataFrame(columns=['index','class','time','state','score','title_ID','method','memory','timeconsume','student_ID'])
from tqdm import tqdm
for i, tid in enumerate(titles_ids):
    print(i, tid)
    page = 0
    while(True):
        page += 1
        print("Page: " + str(page), end='\r')
        url = f"https://acm.sjtu.edu.cn/OnlineJudge/status?page={page}&username=&problem_id={tid}&status=&lang="
        cur_page_df = pd.DataFrame(columns=['index','class','time','state','score','title_ID','method','memory','timeconsume','student_ID'])
        driver.get(url)
        time.sleep(0.5)
        trs = driver.find_elements(By.XPATH, "//*[@id='status']/div[2]/table/tbody/tr")
        if len(trs) == 0:
            break
        for tr in trs:
            tds = tr.find_elements(By.XPATH, ".//td")
            index = tds[0].text
            student_ID = tds[1].find_elements(By.XPATH, ".//span")[0].text
            if not validate_studentID(student_ID):
                continue
            title_ID = "Question_" + str(tid)
            state = tds[3].text
            timeconsume = tds[4].get_attribute("innerText")
            if timeconsume == 'N/A':
                timeconsume = 0
            else:
                timeconsume = int(timeconsume.split(' ')[0])
            memory = tds[5].get_attribute("innerText")
            if memory == 'N/A':
                memory = 0
            else:
                memory = int(memory.split(' ')[0])
            method = "Method_" + tds[6].text
            time_ = tds[7].text
            class_ = "Class_" + str(student_ID[:3])
            score = 1 if state == 'Accepted' else 0
            new_row = {'index':index,'class':class_,'time':time_,'state':state,'score':score,'title_ID':title_ID,'method':method,'memory':memory,'timeconsume':timeconsume,'student_ID':student_ID}
            cur_page_df.loc[len(cur_page_df)] = new_row
        submit_record_df = pd.concat([submit_record_df, cur_page_df], ignore_index=True)

0 2119
1 212691
2 132558
3 214388
4 2142112
5 1088143
6 2158341
7 216085
8 110485
9 217784
10 218523
11 121006
12 11420
13 218888
14 21895
15 22178
16 22184
17 11419
18 223089
19 22326
20 117509
21 224388
22 22442
23 14290


In [97]:
submit_record_df

,index,class,time,state,score,title_ID,method,memory,timeconsume,student_ID
0,456595,Class_523,2024-06-16 23:24:23,Wrong Answer,0,Question_2119,Method_C++,3732,18,523031910388_xujingrui
1,456594,Class_523,2024-06-16 23:22:29,Wrong Answer,0,Question_2119,Method_C++,3960,18,523031910388_xujingrui
2,456593,Class_523,2024-06-16 23:14:50,Wrong Answer,0,Question_2119,Method_C++,3700,18,523031910388_xujingrui
3,456592,Class_523,2024-06-16 23:14:29,Compile Error,0,Question_2119,Method_C++,0,0,523031910388_xujingrui
4,456591,Class_523,2024-06-16 23:14:05,Wrong Answer,0,Question_2119,Method_C++,3672,16,523031910388_xujingrui
...,...,...,...,...,...,...,...,...,...,...
36148,301257,Class_522,2023-05-27 11:20:01,Wrong Answer,0,Question_14290,Method_C++,10824,1009,522031910679_yangzhongkai
36149,301243,Class_522,2023-05-27 11:16:14,Time Limit Exceeded,0,Question_14290,Method_C++,66688,13414,522031910679_yangzhongkai
36150,301242,Class_522,2023-05-27 11:14:52,Time Limit Exceeded,0,Question_14290,Method_C++,66688,13404,522031910679_yangzhongkai
36151,288395,Class_522,2023-05-09 17:15:29,Accepted,1,Question_14290,Method_C++,18064,553,522031910407_chengzicong


In [98]:
submit_record_df.to_csv('./Data_SubmitRecord.csv', index=False)